<a href="https://colab.research.google.com/github/Tomasz-Rac/Modu--4/blob/master/Zadanie_11_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash==2.14.1 werkzeug==2.3.7 flask==2.3.3

# Wczytywanie danych

import pandas as pd
import datetime as dt
import os
import dash as dcc
import dash as html
#import dash.dependencies as Input, Output
from dash.dependencies import Input, Output
import plotly.graph_objs as go

import dash
from dash import dcc, html
from dash.dependencies import Input, Output


class db:
    def __init__(self):
        self.transactions = db.transaction_init()
        self.cc = pd.read_csv(r'db/country_codes.csv', index_col=0)
        self.customers = pd.read_csv(r'db/customers.csv', index_col=0)
        self.prod_info = pd.read_csv(r'db/prod_cat_info.csv')

    @staticmethod
    def transaction_init():
        transactions = pd.DataFrame()
        src = r'db/transactions'
        for filename in os.listdir(src):
          transactions_list = [pd.read_csv(os.path.join(src, filename), index_col=0) for filename in os.listdir(src)]
          transactions = pd.concat(transactions_list, ignore_index=False)
        def convert_dates(x):
            try:
                return dt.datetime.strptime(x, '%d-%m-%Y')
            except:
                return dt.datetime.strptime(x, '%d/%m/%Y')

        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

        return transactions


    def merge(self):
        df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
        .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

        df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
        .set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')

        df = df.join(self.customers.join(self.cc,on='country_code')
        .set_index('customer_Id'),on='cust_id')

        self.merged = df

df = db()
df.merge()

# Budowa podstawowego layoutu

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)


app.layout = html.Div([html.Div([dcc.Tabs(id='tabs',value='tab-1',children=[
                            dcc.Tab(label='Sprzedaż globalna',value='tab-1'),
                            dcc.Tab(label='Produkty',value='tab-2'),
                            dcc.Tab(label='Kanały sprzedaży',value='tab-3'),
                            ]),
                            html.Div(id='tabs-content')
                    ],style={'width':'80%','margin':'auto'})],
                    style={'height':'100%'})

#0 nowo dodane
@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return render_tab(df.merged)
    elif tab == 'tab-2':
        return render_tab2(df.merged)
    elif tab == 'tab-3':
        return render_tab3(df.merged)




#0 nowo dodane


if __name__ == '__main__':
    app.run_server(debug=True)


# Sprzedaż globalna

def render_tab(df):

    layout = html.Div([html.H1('Sprzedaż globalna',style={'text-align':'center'}),
                        html.Div([dcc.DatePickerRange(id='sales-range',
                        start_date=df['tran_date'].min(),
                        end_date=df['tran_date'].max(),
                        display_format='YYYY-MM-DD')],style={'width':'100%','text-align':'center'}),
                        html.Div([html.Div([dcc.Graph(id='bar-sales')],style={'width':'50%'}),
                        html.Div([dcc.Graph(id='choropleth-sales')],style={'width':'50%'})],style={'display':'flex'})
                        ])

    return layout

# Produkty

def render_tab2(df):

    grouped = df[df['total_amt']>0].groupby('prod_cat')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index,values=grouped.values)],layout=go.Layout(title='Udział grup produktów w sprzedaży'))

    layout = html.Div([html.H1('Produkty',style={'text-align':'center'}),

                        html.Div([html.Div([dcc.Graph(id='pie-prod-cat',figure=fig)],style={'width':'50%'}),
                        html.Div([dcc.Dropdown(id='prod_dropdown',
                                    options=[{'label':prod_cat,'value':prod_cat} for prod_cat in df['prod_cat'].unique()],
                                    value=df['prod_cat'].unique()[0]),
                                    dcc.Graph(id='barh-prod-subcat')],style={'width':'50%'})],style={'display':'flex'}),
                                    html.Div(id='temp-out')
                        ])

    return layout

#3 Kanały sprzedaży
def render_tab3(df):
    layout = html.Div([
        html.H1("Kanały sprzedaży", style={'textAlign': 'center'}),


        html.Div([
            html.Div([
                dcc.Dropdown(
                    id='store_dropdown',
                    options=[{'label': store, 'value': store} for store in df['Store_type'].dropna().unique()],
                    value=df['Store_type'].dropna().unique()[0]
                ),
                dcc.Graph(id='barh-store-gender')
            ], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])

    return layout






# Callbacks


def tab1_bar_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby([pd.Grouper(key='tran_date',freq='M'),'Store_type'])['total_amt'].sum().round(2).unstack()

    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(x=grouped.index,y=grouped[col],name=col,hoverinfo='text',
        hovertext=[f'{y/1e3:.2f}k' for y in grouped[col].values]))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(title='Przychody',barmode='stack',legend=dict(x=0,y=-0.5)))

    return fig

@app.callback(Output('bar-sales', 'figure'),
              [Input('sales-range', 'start_date'),
               Input('sales-range', 'end_date')])
def update_bar_sales(start_date, end_date):
    return tab1_bar_sales(start_date, end_date)

@app.callback(Output('choropleth-sales','figure'),
            [Input('sales-range','start_date'),Input('sales-range','end_date')])
def tab1_choropleth_sales(start_date,end_date):

    truncated = df.merged[(df.merged['tran_date']>=start_date)&(df.merged['tran_date']<=end_date)]
    grouped = truncated[truncated['total_amt']>0].groupby('country')['total_amt'].sum().round(2)

    trace0 = go.Choropleth(colorscale='Viridis',reversescale=True,
                            locations=grouped.index,locationmode='country names',
                            z = grouped.values, colorbar=dict(title='Sales'))
    data = [trace0]
    fig = go.Figure(data=data,layout=go.Layout(title='Mapa',geo=dict(showframe=False,projection={'type':'natural earth'})))

    return fig

# tab2 callbacks
@app.callback(Output('barh-prod-subcat','figure'),
            [Input('prod_dropdown','value')])
def tab2_barh_prod_subcat(chosen_cat):

    grouped = df.merged[(df.merged['total_amt']>0)&(df.merged['prod_cat']==chosen_cat)].pivot_table(index='prod_subcat',columns='Gender',values='total_amt',aggfunc='sum').assign(_sum=lambda x: x['F']+x['M']).sort_values(by='_sum').round(2)

    traces = []
    for col in ['F','M']:
        traces.append(go.Bar(x=grouped[col],y=grouped.index,orientation='h',name=col))

    data = traces
    fig = go.Figure(data=data,layout=go.Layout(barmode='stack',margin={'t':20,}))
    return fig


#3

@app.callback(Output('barh-store-gender', 'figure'),
              [Input('store_dropdown', 'value')])
def update_gender_qty_by_day(store_type):
    df_filtered = df.merged.copy()
    df_filtered['weekday'] = df_filtered['tran_date'].dt.day_name()

    grouped = df_filtered[df_filtered['Store_type'] == store_type]
    grouped = grouped.pivot_table(index='weekday', columns='Gender', values='Qty', aggfunc='sum').fillna(0)


    weekday_mapping = {
        'Monday': 'Poniedziałek',
        'Tuesday': 'Wtorek',
        'Wednesday': 'Środa',
        'Thursday': 'Czwartek',
        'Friday': 'Piątek',
        'Saturday': 'Sobota',
        'Sunday': 'Niedziela'
    }

    weekday_order = list(weekday_mapping.keys())
    grouped = grouped.reindex(weekday_order)
    grouped.index = grouped.index.map(weekday_mapping)





    traces = []
    for gender in ['F', 'M']:
        if gender in grouped.columns:
            traces.append(go.Bar(
                x=grouped[gender],
                y=grouped.index,
                name=gender,
                orientation='h'
            ))

    fig = go.Figure(data=traces)
    fig.update_layout(
        barmode='stack',
        title=f'Ilość sprzedaży wg dnia tygodnia i udział płci – {store_type}',
        xaxis_title='Ilość',
        yaxis_title='Dzień tygodnia',
        margin=dict(t=40)
    )
    return fig




#działający kod !




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


<IPython.core.display.Javascript object>